In [1]:
from datetime import datetime
import dask.array as da
import os
import time
import h5py
import numpy as np
import xarray as xr
import downscale as down
import matplotlib.pyplot as plt

import warnings

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

In [2]:
tmpa_dir = os.path.join('..', 'data', 'tmpa_conus_data')
inputfile = os.path.join(tmpa_dir, 'data_tmpa_3h.hdf5')
outfile = os.path.join(tmpa_dir, 'data_tmpa_3h_cut.nc')

In [3]:
ncf = xr.open_dataset(inputfile, engine="h5netcdf",phony_dims='sort')

tmpalat = ncf['lat'][:].values
tmpalon = ncf['lon'][:].values
dates_int = ncf['dates'][:].values
hours_int = ncf['hours'][:].values
dset = ncf['prcp'].values

del ncf

In [4]:
min_lon = -100
max_lon = -90
min_lat = 30
max_lat = 36

In [5]:
x = da.from_array(dset, chunks=(6, 6, 300))
dates = [datetime.strptime(str(integd)+str(inthour), '%Y%m%d%H') for integd, inthour in zip(dates_int, hours_int)]

xrs0 = xr.DataArray(x,  coords={'lon':tmpalon, 'lat':tmpalat, 'time':dates}, dims=('lon', 'lat', 'time'))
xrs0 = xrs0.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

In [6]:
pre = np.array(xrs0.data)
pre = np.swapaxes(pre,0,2)
pre.shape

(2433, 24, 40)

In [7]:
PRE_xr = xr.Dataset(data_vars={"PRE": (("time", "lat", "lon"), pre)},
                    coords={'time': dates, 'lat': xrs0['lat'].values, 'lon': xrs0['lon'].values},
                    attrs=dict(description="TMPA 3h"))
PRE_xr.PRE.attrs["units"] = "millimeter"
PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lon.attrs["units"] = "degrees_east"

In [8]:
print(f'Export PRE data to {outfile}')
PRE_xr.to_netcdf(outfile)

Export PRE data to ../data/tmpa_conus_data/data_tmpa_3h_cut.nc


In [9]:
PRE_xr

<xarray.Dataset> Size: 9MB
Dimensions:  (time: 2433, lat: 24, lon: 40)
Coordinates:
  * time     (time) datetime64[ns] 19kB 2008-01-01 ... 2018-10-31T21:00:00
  * lat      (lat) float32 96B 30.12 30.38 30.62 30.88 ... 35.38 35.62 35.88
  * lon      (lon) float32 160B -99.88 -99.62 -99.38 ... -90.62 -90.38 -90.12
Data variables:
    PRE      (time, lat, lon) float32 9MB 0.0 0.0 0.0 0.0 ... 0.0 4.871 0.0 0.0
Attributes:
    description:  TMPA 3h